In [7]:
#Extract Frames from the Video: Use Python and OpenCV to extract frames from the video
import cv2
import os

# Function to extract frames from a video and save them
def extract_frames(video_path, output_dir, basename):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    # Loop to read and save each frame
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_name = os.path.join(output_dir, f'{basename}_frame_{frame_count}.jpg')
        cv2.imwrite(frame_name, frame)
        frame_count += 1

    cap.release()
    print(f"Extracted {frame_count} frames from {video_path}")
    return frame_count

In [17]:
import scipy.io

# Load labels from .mat file
def load_labels_from_mat(mat_file):
    mat_data = scipy.io.loadmat(mat_file)
    print(mat_data.keys())
    gTruth = mat_data.get('gTruth', None)  # Replace with actual key if different
    if gTruth is None:
        raise ValueError("Ground truth data not found in the .mat file")

    # Extract label data (adjust according to structure of gTruth)
    # Assuming 'LabelData' contains bounding boxes and class labels
    label_data = gTruth['LabelData']  # adjust key based on your .mat file structure
    return label_data


In [18]:
#convert labels to yolo format
def convert_to_yolo_format(bbox, img_width, img_height):
    # bbox format: [x_min, y_min, width, height]
    x_center = (bbox[0] + bbox[2] / 2) / img_width
    y_center = (bbox[1] + bbox[3] / 2) / img_height
    width = bbox[2] / img_width
    height = bbox[3] / img_height
    return [x_center, y_center, width, height]

In [19]:
#save Frames and YOLO labels
import os

# Save frames and YOLO labels for a video
def save_yolo_labels(frame_count, labels, output_image_dir, output_label_dir, basename, class_mapping, img_width, img_height):
    os.makedirs(output_image_dir, exist_ok=True)
    os.makedirs(output_label_dir, exist_ok=True)

    for i in range(frame_count):
        # Define frame and label paths
        frame_name = f'{basename}_frame_{i}.jpg'
        frame_path = os.path.join(output_image_dir, frame_name)
        label_path = os.path.join(output_label_dir, f'{basename}_frame_{i}.txt')
        
        with open(label_path, 'w') as f:
            # Loop through each bounding box for the frame
            for label in labels[i]:  # assuming labels is a list of bbox for each frame
                class_name = label['ClassName']
                class_id = class_mapping[class_name]
                bbox = label['BoundingBox']
                
                # Convert bbox to YOLO format
                yolo_bbox = convert_to_yolo_format(bbox, img_width, img_height)
                
                # Write YOLO formatted label to file
                f.write(f"{class_id} " + " ".join(map(str, yolo_bbox)) + "\n")
    print(f"Saved YOLO labels for {frame_count} frames")


In [20]:
#process all videos and labels
# Define paths and class mapping
import os 
import glob

data_dir = os.path.join("..", "Drone-detection-dataset/Data/Video_IR")  # Relative path to data directory
video_dir = data_dir
label_dir = data_dir

output_image_dir = os.path.join("..", "training_data", "images")
output_label_dir = os.path.join("..", "training_data", "labels")
class_mapping = {"AIRPLANE": 0, "BIRD": 1, "DRONE": 2, "HELICOPTER": 3}

# Process each video and its corresponding .mat file
for video_file in os.listdir(video_dir):
    if video_file.endswith('.mp4'):
        basename = os.path.splitext(video_file)[0]
        video_path = os.path.join(video_dir, video_file)
        mat_file = os.path.join(label_dir, f"{basename}_LABELS.mat")

        # Extract frames from video
        frame_count = extract_frames(video_path, output_image_dir, basename)

        # Load labels from .mat file
        labels = load_labels_from_mat(mat_file)

        # Assuming all frames have same resolution
        cap = cv2.VideoCapture(video_path)
        ret, frame = cap.read()
        img_height, img_width = frame.shape[:2]
        cap.release()

        # Save images and YOLO labels
        save_yolo_labels(frame_count, labels, output_image_dir, output_label_dir, basename, class_mapping, img_width, img_height)

print("Completed processing all videos and labels.")

Extracted 301 frames from ../Drone-detection-dataset/Data/Video_IR/IR_AIRPLANE_038.mp4
dict_keys(['__header__', '__version__', '__globals__', 'None', '__function_workspace__'])


ValueError: Ground truth data not found in the .mat file

In [23]:
import scipy.io
import os

# Load labels from .mat file
def load_labels_from_mat(mat_file):
    mat_data = scipy.io.loadmat(mat_file)
    print(mat_data.keys())

    # print(mat_data['__function_workspace__'])
    # print(mat_data['__header__'])
    # print(mat_data['__version__'])
    # print(mat_data['__globals__'])
    print(mat_data['None'])

    gTruth = mat_data.get('gTruth', None)  # Replace with actual key if different
    if gTruth is None:
        raise ValueError("Ground truth data not found in the .mat file")

    # Extract label data (adjust according to structure of gTruth)
    # Assuming 'LabelData' contains bounding boxes and class labels
    label_data = gTruth['LabelData']  # adjust key based on your .mat file structure
    return label_data

data_dir = os.path.join("..", "Drone-detection-dataset/Data/Video_IR/IR_AIRPLANE_001_LABELS.mat")  # Relative path to data directory

load_labels_from_mat(data_dir)



dict_keys(['__header__', '__version__', '__globals__', 'None', '__function_workspace__'])
[(b'gTruth', b'MCOS', b'groundTruth', array([[3707764736],
        [         2],
        [         1],
        [         1],
        [         1],
        [         5]], dtype=uint32))                     ]


ValueError: Ground truth data not found in the .mat file

ParseError: Got type 1, expected 5 (miINT32)